### Generate trial structure and write json files
- The experiment consists of a block of 20 practice trials and 12 blocks of the actual experimental trials (32 trials each). The 12 conditions are 4 (Dist-RE combinations) * 3 durations (34, 84, 167 ms).
- image_df is the dataframe with information about the percentile of distinctiveness and reconstruction error for images (both targets and fillers).

In [26]:
import cv2
import os
import pandas as pd
import numpy as np
import itertools
import random
import shutil
import json

In [27]:
output_dir = './Experiment_Files'
target_img_dir = './Stimuli/Targets'
filler_img_dir = './Stimuli/Fillers'
img_info_dir = './Image_Info/images_for_trial_new.csv'
labjs_dir = './RSVP_Exp/embedded'
image_df = pd.read_csv(img_info_dir) # A dataframe containing 

In [28]:
# Define a function for writing json images
def write_json_file(curr_df, prac_df):
    
    all_blocks_dict = dict()
    
    # Practice block
    # Create the trial info first
    curr_block_trial_info = []
    curr_block_files = dict()
    
    for curr_row in prac_df.iterrows():

            target_img_1 = "{}_{}.jpg".format(curr_row[1]['target_img_0'].split('/')[1][:-4], curr_row[1]['target_img_0'][0])
            target_img_2 = "{}_{}.jpg".format(curr_row[1]['target_img_1'].split('/')[1][:-4], curr_row[1]['target_img_1'][0])
            target_img_3 = "{}_{}.jpg".format(curr_row[1]['target_img_2'].split('/')[1][:-4], curr_row[1]['target_img_2'][0])
            target_img_4 = "{}_{}.jpg".format(curr_row[1]['target_img_3'].split('/')[1][:-4], curr_row[1]['target_img_3'][0])
            target_img_5 = "{}_{}.jpg".format(curr_row[1]['target_img_4'].split('/')[1][:-4], curr_row[1]['target_img_4'][0])
            target_img_6 = "{}_{}.jpg".format(curr_row[1]['target_img_5'].split('/')[1][:-4], curr_row[1]['target_img_5'][0])
            test_img = "{}_{}.jpg".format(curr_row[1]['test_img'].split('/')[1][:-4], curr_row[1]['test_img'][0])



            curr_trial_dict = {
                "TrialType": curr_row[1]['TrialType'],
                "is_probe": curr_row[1]['ifProbe'],
                "probesize0" : curr_row[1]['ProbeSize_0'], 
                "probesize1" : curr_row[1]['ProbeSize_1'], 
                "probesize2" : curr_row[1]['ProbeSize_2'], 
                "probesize3" : curr_row[1]['ProbeSize_3'], 
                "probesize4" : curr_row[1]['ProbeSize_4'], 
                "probesize5" : curr_row[1]['ProbeSize_5'], 
                "probe_top": curr_row[1]['top'],
                "probe_left": curr_row[1]['left'],
                
              "target_img_1": target_img_1,
              "target_img_2": target_img_2,
              "target_img_3": target_img_3,
              "target_img_4": target_img_4,
              "target_img_5": target_img_5,
              "target_img_6": target_img_6,
              "test_img": test_img
            }

            curr_block_trial_info.append(curr_trial_dict)

            # generate all the files
            curr_block_files[target_img_1] = 'embedded/{}'.format(target_img_1)
            curr_block_files[target_img_2] = 'embedded/{}'.format(target_img_2)
            curr_block_files[target_img_3] = 'embedded/{}'.format(target_img_3)
            curr_block_files[target_img_4] = 'embedded/{}'.format(target_img_4)
            curr_block_files[target_img_5] = 'embedded/{}'.format(target_img_5)
            curr_block_files[target_img_6] = 'embedded/{}'.format(target_img_6)

            if curr_row[1]['TrialType'] == 'NO':
                 curr_block_files[test_img] = 'embedded/{}'.format(test_img)


#     print(len(curr_block_trial_info))
    all_blocks_dict['prac_trial_info'] = curr_block_trial_info
    all_blocks_dict['prac_files'] = curr_block_files

    
#     for curr_block in range(8):
    for curr_block in range(12):

        curr_block_df = curr_df[curr_df['blockNum']==curr_block].reset_index(drop=True)

        # Create the trial info first
        curr_block_trial_info = []
        curr_block_files = dict()

        curr_frameNum = curr_block_df['FrameNum'].values[-1]
        if curr_frameNum == 5:
            curr_block_dur = "84"
        elif curr_frameNum == 2:
            curr_block_dur = "34"
        elif curr_frameNum == 10:
            curr_block_dur = "167"

        curr_seq_num = curr_block_df['sub'].values[0]

        for curr_row in curr_block_df.iterrows():


            target_img_1 = "{}_{}.jpg".format(curr_row[1]['target_img_0'].split('/')[1][:-4], curr_row[1]['target_img_0'][0])
            target_img_2 = "{}_{}.jpg".format(curr_row[1]['target_img_1'].split('/')[1][:-4], curr_row[1]['target_img_1'][0])
            target_img_3 = "{}_{}.jpg".format(curr_row[1]['target_img_2'].split('/')[1][:-4], curr_row[1]['target_img_2'][0])
            target_img_4 = "{}_{}.jpg".format(curr_row[1]['target_img_3'].split('/')[1][:-4], curr_row[1]['target_img_3'][0])
            target_img_5 = "{}_{}.jpg".format(curr_row[1]['target_img_4'].split('/')[1][:-4], curr_row[1]['target_img_4'][0])
            target_img_6 = "{}_{}.jpg".format(curr_row[1]['target_img_5'].split('/')[1][:-4], curr_row[1]['target_img_5'][0])
            test_img = "{}_{}.jpg".format(curr_row[1]['test_img'].split('/')[1][:-4], curr_row[1]['test_img'][0])


            curr_trial_dict = {
                "TrialType": curr_row[1]['TrialType'],
                'Condition': 'Dist_{}_RE_{}_{}'.format(curr_row[1]['Dist'], curr_row[1]['RE'], curr_frameNum),
                'sub_seq': int(curr_seq_num),
                "is_probe": curr_row[1]['ifProbe'],
                "probesize0" : curr_row[1]['ProbeSize_0'], 
                "probesize1" : curr_row[1]['ProbeSize_1'], 
                "probesize2" : curr_row[1]['ProbeSize_2'], 
                "probesize3" : curr_row[1]['ProbeSize_3'], 
                "probesize4" : curr_row[1]['ProbeSize_4'], 
                "probesize5" : curr_row[1]['ProbeSize_5'], 
                "probe_left": curr_row[1]['left'],
                "probe_top": curr_row[1]['top'],
            

              "target_img_1": target_img_1,
              "target_img_2": target_img_2,
              "target_img_3": target_img_3,
              "target_img_4": target_img_4,
              "target_img_5": target_img_5,
              "target_img_6": target_img_6,
              "test_img": test_img
            }

            curr_block_trial_info.append(curr_trial_dict)

            # generate all the files
            curr_block_files[target_img_1] = 'embedded/{}'.format(target_img_1)
            curr_block_files[target_img_2] = 'embedded/{}'.format(target_img_2)
            curr_block_files[target_img_3] = 'embedded/{}'.format(target_img_3)
            curr_block_files[target_img_4] = 'embedded/{}'.format(target_img_4)
            curr_block_files[target_img_5] = 'embedded/{}'.format(target_img_5)
            curr_block_files[target_img_6] = 'embedded/{}'.format(target_img_6)

            if curr_row[1]['TrialType'] == 'NO':
                 curr_block_files[test_img] = 'embedded/{}'.format(test_img)
            

        all_blocks_dict['block{}_trial_info'.format(curr_block+1)] = curr_block_trial_info
        all_blocks_dict['block{}_files'.format(curr_block+1)] = curr_block_files
        all_blocks_dict['block{}_dur'.format(curr_block+1)] = curr_block_dur

    json_dir = './RSVP_Exp/exp_seqs'
    output_dir = os.path.join(json_dir, '{}_32trials_3times.json'.format(curr_seq_num))
    with open(output_dir , 'w') as fp:
        json.dump(all_blocks_dict, fp)

    fp.close()


In [29]:
image_df['same_dir'] = image_df['Distinctiveness percentile'] + image_df['RE percentile'] 

In [30]:
image_df['diff_dir'] = image_df['Distinctiveness percentile'] - image_df['RE percentile'] 

In [31]:
# Select the most extreme 48 images to be used as targets
percentile_type = ['Small','Large']
count = 0
for curr_comb in itertools.product(percentile_type, percentile_type):
    curr_df = image_df[(image_df['Distinctiveness type']==curr_comb[0])&
                      (image_df['RE type']==curr_comb[1])]
    if (curr_comb[0]=='Large') & (curr_comb[1]=='Large'):
        curr_df = curr_df.sort_values(by='same_dir').reset_index(drop=True)
        selected_df = curr_df.iloc[-48:]
    elif (curr_comb[0]=='Small') & (curr_comb[1]=='Small'):
        curr_df = curr_df.sort_values(by='same_dir').reset_index(drop=True)
        selected_df = curr_df.iloc[:48]
    elif (curr_comb[0]=='Large') & (curr_comb[1]=='Small'):
        curr_df = curr_df.sort_values(by='diff_dir').reset_index(drop=True)
        selected_df = curr_df.iloc[-48:]
    elif (curr_comb[0]=='Small') & (curr_comb[1]=='Large'):
        curr_df = curr_df.sort_values(by='diff_dir').reset_index(drop=True)
        selected_df = curr_df.iloc[:48]  
    
    if count == 0:
        selected_target_df = selected_df
        count += 1
    else:
        selected_target_df = pd.concat([selected_target_df, selected_df]).reset_index(drop=True)

In [32]:
# Move some more fillers to use for practice
all_fillers = list(range(1, 8043))
remaining_fillers = [k for k in all_fillers if k not in image_df[image_df['Image type']=='Filler']['Image index'].values]
sampled_fillers = remaining_fillers[:160]
for curr_prac_img in sampled_fillers:
    orig = os.path.join(filler_img_dir,'{}.jpg'.format(curr_prac_img))
    if os.path.isfile(orig):
        #print('add')
        dest = os.path.join(labjs_dir,'{}_F.jpg'.format(curr_prac_img))
        shutil.move(orig,dest)

In [33]:
for curr_row in image_df.iterrows():
    
    curr_image_type = curr_row[1]['Image type']
    curr_image_ind = curr_row[1]['Image index']
    
    if curr_image_type == 'Filler':
        orig = os.path.join(filler_img_dir, '{}.jpg'.format(curr_image_ind))
        dest = os.path.join(labjs_dir,'{}_F.jpg'.format(curr_image_ind))
        shutil.copy(orig,dest)
    elif curr_image_type == 'Target':
        orig = os.path.join(target_img_dir, '{}.jpg'.format(curr_image_ind))
        dest = os.path.join(labjs_dir,'{}_T.jpg'.format(curr_image_ind))
        shutil.copy(orig,dest)

In [34]:
# Generate the json files in desired format

In [35]:
TrialType_dict = {1:'YES', 0:'NO'}
ifProbe_dict = {1:'YES', 0:'NO'}

In [36]:
for curr_sub in range(25):
    
    sampled_fillers_prac = sampled_fillers.copy()
    # Set up practice
    nTrials_prac = 20
    nTarget_Trials_prac = int(nTrials_prac/2)

    
    for curr_Trial in range(nTrials_prac):
        # Generate trial sequences
        target_vect = np.ones((nTarget_Trials_prac, 2))
        filler_vect = np.zeros((nTarget_Trials_prac, 2))
    
        target_vect[:,1] = np.random.choice([2,3,4,5], nTarget_Trials_prac)

    curr_pracTrials = np.vstack((target_vect, filler_vect))
    curr_prac_df = pd.DataFrame(curr_pracTrials, columns = ['TrialType', 'Pos'])

    curr_prac_df = curr_prac_df.sample(frac=1)
    curr_prac_df['FrameNum'] = 5
    curr_prac_df['Dist'] = np.nan
    curr_prac_df['RE'] = np.nan
    curr_prac_df['trialNum'] = np.array(range(nTrials_prac))
    curr_prac_df['ifProbe'] = 0
    curr_prac_df['left'] = 0 
    curr_prac_df['top'] = 0 

    curr_prac_df['ProbeSize_0'] = 0
    curr_prac_df['ProbeSize_1'] = 0
    curr_prac_df['ProbeSize_2'] = 0
    curr_prac_df['ProbeSize_3'] = 0
    curr_prac_df['ProbeSize_4'] = 0
    curr_prac_df['ProbeSize_5'] = 0
    
    
    for curr_target_img_ind in range(6):
        curr_prac_df['target_img_{}'.format(curr_target_img_ind)] = ''
        curr_prac_df['test_img'] = ''
    
        curr_prac_df['ifProbe'] = 0
        curr_prac_df['left'] = 0 
        curr_prac_df['top'] = 0

    for curr_row in curr_prac_df.iterrows():
        curr_trial = int(curr_row[1]['trialNum'])
        curr_framenum = int(curr_row[1]['FrameNum'])
        curr_trial_type = int(curr_row[1]['TrialType'])
        curr_Pos = int(curr_row[1]['Pos'])

        #img_seq = []
        if curr_trial_type == 0:


            for curr_img_ind in range(6):
                curr_img_dir = os.path.join('Filler', '{}.jpg'.format(sampled_fillers_prac[0]))
                sampled_fillers_prac.pop(0)
                curr_prac_df.loc[curr_row[0], 'target_img_{}'.format(curr_img_ind)] = curr_img_dir
                curr_prac_df.loc[curr_row[0], 'ProbeSize_{}'.format(curr_img_ind)] = 0

            
            # pick a random test image
            curr_img_dir = os.path.join('Filler', '{}.jpg'.format(sampled_fillers_prac[0]))
            sampled_fillers_prac.pop(0)
            curr_prac_df.loc[curr_row[0], 'test_img'] = curr_img_dir

            curr_prac_df.loc[curr_row[0], 'left'] = 0
            curr_prac_df.loc[curr_row[0], 'top'] = 0

        elif  curr_trial_type == 1:
            random_test = random.randint(1,4) 
           
            for curr_img_ind in range(6):
                if curr_img_ind != curr_Pos-1:
                    curr_img_dir = os.path.join('Filler', '{}.jpg'.format(sampled_fillers_prac[0]))
                    sampled_fillers_prac.pop(0)
                else:
                    curr_img_dir = os.path.join('Filler', '{}.jpg'.format(sampled_fillers_prac[0]))
                    sampled_fillers_prac.pop(0)
                    curr_target_img_dir = curr_img_dir

                    if 1 <= random_test <= 3: 
                        curr_prac_df.loc[curr_row[0], 'ProbeSize_{}'.format(curr_img_ind)] = 10
                        curr_prac_df.loc[curr_row[0], 'ifProbe'] = 1
                    elif random_test == 4: 
                        curr_prac_df.loc[curr_row[0], 'ProbeSize_{}'.format(curr_img_ind)] = 0
                    
                curr_prac_df.loc[curr_row[0], 'target_img_{}'.format(curr_img_ind)] = curr_img_dir

               

            curr_prac_df.loc[curr_row[0], 'test_img'] = curr_target_img_dir

            if 1 <= random_test <= 3: 
                random_loc = random.randint(1,4)
                if random_loc == 1:
                    curr_prac_df.loc[curr_row[0], 'left'] = 100
                    curr_prac_df.loc[curr_row[0], 'top'] = -90
                elif random_loc == 2: 
                    curr_prac_df.loc[curr_row[0], 'left'] = 100
                    curr_prac_df.loc[curr_row[0], 'top'] = 90
                elif random_loc == 3: 
                    curr_prac_df.loc[curr_row[0], 'left'] = -100
                    curr_prac_df.loc[curr_row[0], 'top'] = -90
                elif random_loc == 4: 
                    curr_prac_df.loc[curr_row[0], 'left']= -100
                    curr_prac_df.loc[curr_row[0], 'top'] = 90

            elif random_test == 4: 
                curr_prac_df.loc[curr_row[0], 'ifProbe'] = 0 
                curr_prac_df.loc[curr_row[0], 'left'] = 0
                curr_prac_df.loc[curr_row[0], 'top'] = 0 


            
    curr_prac_df['TrialType'] = curr_prac_df['TrialType'].map(TrialType_dict)
    curr_prac_df['ifProbe'] = curr_prac_df['ifProbe'].map(ifProbe_dict)
    
    # Read in the image info
    image_df = pd.read_csv(img_info_dir)
    

    # Set up all the trial types and randomize order
    nTrials_per_block = 32;
    nTrials_per_pos = nTrials_per_block/2/4;
    nTarget_Trials = int(nTrials_per_block/2);
    list_of_pos = [2,3,4,5];# Where can the target image be in the sequence of 6 images
    list_of_times = [2, 5, 10]; # specificy in the unit of framerate
    list_of_dist = [0,1];
    list_of_RE = [0,1];
    percentile_type = ['Small','Large']
    all_block_types = list(itertools.product(list_of_times, list_of_dist, list_of_RE))
    random.shuffle(all_block_types)

    frame = cv2.imread(os.path.join(target_img_dir, '1.jpg'))
    height, width, layers = frame.shape

    # Generate trial sequences
    for curr_block_ind, curr_block_info in enumerate(all_block_types):
        curr_enc_time = curr_block_info[0]
        curr_dist = percentile_type[curr_block_info[1]]
        curr_RE =  percentile_type[curr_block_info[2]]

        # Generate trial sequences
        target_vect = np.ones((nTarget_Trials, 2))
        filler_vect = np.zeros((nTarget_Trials, 2))

        for curr_pos_ind, curr_pos in enumerate(list_of_pos):
            target_vect[int(curr_pos_ind*nTrials_per_pos):int((curr_pos_ind+1)*nTrials_per_pos),1] = curr_pos

        curr_blockTrials = np.vstack((target_vect, filler_vect));

        curr_block_df = pd.DataFrame(curr_blockTrials, columns = ['TrialType', 'Pos'])

        curr_block_df = curr_block_df.sample(frac=1)
        curr_block_df['FrameNum'] = curr_enc_time
        curr_block_df['Dist'] = curr_dist
        curr_block_df['RE'] = curr_RE
        curr_block_df['blockNum'] = curr_block_ind
        curr_block_df['trialNum'] = np.array(range(nTrials_per_block))

        if curr_block_ind == 0:
            all_blocks_df = curr_block_df
        else:
            all_blocks_df = pd.concat([all_blocks_df, curr_block_df]).reset_index(drop=True)

    for curr_target_img_ind in range(6):
        all_blocks_df['target_img_{}'.format(curr_target_img_ind)] = ''
    all_blocks_df['test_img'] = ''

    # Separate all the images
    LDist_LRE = list(selected_target_df[(selected_target_df['Distinctiveness type']=='Large') & (selected_target_df['RE type']=='Large')]['Image index'].values)
    LDist_LRE = [str(k) for k in LDist_LRE]
    random.shuffle(LDist_LRE)
    SDist_SRE = list(selected_target_df[(selected_target_df['Distinctiveness type']=='Small') & (selected_target_df['RE type']=='Small')]['Image index'].values)
    SDist_SRE = [str(k) for k in SDist_SRE]
    random.shuffle(SDist_SRE)
    LDist_SRE = list(selected_target_df[(selected_target_df['Distinctiveness type']=='Large') & (selected_target_df['RE type']=='Small')]['Image index'].values)
    LDist_SRE = [str(k) for k in LDist_SRE]
    random.shuffle(LDist_SRE)
    SDist_LRE = list(selected_target_df[(selected_target_df['Distinctiveness type']=='Small') & (selected_target_df['RE type']=='Large')]['Image index'].values)
    SDist_LRE = [str(k) for k in SDist_LRE]
    random.shuffle(SDist_LRE)
    fillers =  list(image_df[image_df['Image type']=='Filler' ]['Image index'].values)
    fillers = [str(k) for k in fillers]
    random.shuffle(fillers)

   
    all_blocks_df['ifProbe'] = 0
    all_blocks_df['ProbeSize_0'] = 0
    all_blocks_df['ProbeSize_1'] = 0
    all_blocks_df['ProbeSize_2'] = 0
    all_blocks_df['ProbeSize_3'] = 0
    all_blocks_df['ProbeSize_4'] = 0
    all_blocks_df['ProbeSize_5'] = 0
    all_blocks_df['left'] = 0 
    all_blocks_df['top'] = 0


    for curr_row in all_blocks_df.iterrows():
        curr_block = int(curr_row[1]['blockNum'])
        curr_trial = int(curr_row[1]['trialNum'])
        curr_framenum = int(curr_row[1]['FrameNum'])
        curr_trial_type = int(curr_row[1]['TrialType'])
        curr_dist = curr_row[1]['Dist']
        curr_RE = curr_row[1]['RE']
        curr_Pos = int(curr_row[1]['Pos'])
        all_blocks_df.loc[curr_row[0], 'left'] = 0 
        all_blocks_df[curr_row[0], 'top'] = 0
        
    
        #img_seq = []
        if curr_trial_type == 0:

            for curr_img_ind in range(6):
                all_blocks_df.loc[curr_row[0], 'ProbeSize_{}'.format(curr_img_ind)] = 0
                curr_img_dir = os.path.join('Filler', '{}.jpg'.format(fillers[0]))
                fillers.pop(0)
                all_blocks_df.loc[curr_row[0], 'target_img_{}'.format(curr_img_ind)] = curr_img_dir

            # pick a random test image
            curr_img_dir = os.path.join('Filler', '{}.jpg'.format(fillers[0]))
            fillers.pop(0)
            all_blocks_df.loc[curr_row[0], 'test_img'] = curr_img_dir

            all_blocks_df.loc[curr_row[0], 'ifProbe'] = 0 
            all_blocks_df.loc[curr_row[0], 'left'] = 0 
            all_blocks_df.loc[curr_row[0], 'top'] = 0

        # if curr_trial_type ==1, pick a target image and also set the probe thing equal to true
        elif  curr_trial_type == 1:
            random_test = random.randint(1,4) 
            
            for curr_img_ind in range(6): 
                if curr_img_ind != curr_Pos-1:
                    curr_img_dir = os.path.join('Filler', '{}.jpg'.format(fillers[0]))
                    fillers.pop(0)

                else:

                    # Separate by target type
                    if (curr_dist == 'Large') & (curr_RE == 'Large'):
                        curr_img_dir = os.path.join('Target', '{}.jpg'.format(LDist_LRE[0]))

                        LDist_LRE.pop(0)
                    elif (curr_dist == 'Small') & (curr_RE== 'Small'):
                        curr_img_dir = os.path.join('Target', '{}.jpg'.format(SDist_SRE[0]))
                        SDist_SRE.pop(0)
                    elif (curr_dist == 'Small') & (curr_RE == 'Large'):
                        curr_img_dir = os.path.join('Target', '{}.jpg'.format(SDist_LRE[0]))
                        SDist_LRE.pop(0)
                    elif (curr_dist == 'Large') & (curr_RE == 'Small'):
                        curr_img_dir = os.path.join('Target', '{}.jpg'.format(LDist_SRE[0]))
                        LDist_SRE.pop(0)
                    curr_target_img_dir = curr_img_dir

                    if 1 <= random_test <= 3:
                        all_blocks_df.loc[curr_row[0], 'ProbeSize_{}'.format(curr_img_ind)] = 10
                        all_blocks_df.loc[curr_row[0], 'ifProbe'] = 1
                    else:
                        all_blocks_df.loc[curr_row[0], 'ProbeSize_{}'.format(curr_img_ind)] = 0

                all_blocks_df.loc[curr_row[0], 'target_img_{}'.format(curr_img_ind)] = curr_img_dir


                   
                
                # if this is a target trial, then randomize if there is a probe and the probe's location 
                if 1 <= random_test <= 3:
                    random_loc = random.randint(1,4)
                    if random_loc == 1:
                        all_blocks_df.loc[curr_row[0], 'left'] = 100
                        all_blocks_df.loc[curr_row[0], 'top'] = 90
                    elif random_loc == 2: 
                        all_blocks_df.loc[curr_row[0], 'left'] = 100
                        all_blocks_df.loc[curr_row[0], 'top'] = -90
                    elif random_loc == 3: 
                        all_blocks_df.loc[curr_row[0], 'left'] = -100
                        all_blocks_df.loc[curr_row[0], 'top'] = -90
                    elif random_loc == 4: 
                        all_blocks_df.loc[curr_row[0], 'left'] = -100
                        all_blocks_df.loc[curr_row[0], 'top'] = 90

                elif random_test == 4: 
                    all_blocks_df.loc[curr_row[0], 'ifProbe'] = 0 
                    all_blocks_df.loc[curr_row[0], 'left'] = 0 
                    all_blocks_df.loc[curr_row[0], 'top'] = 0


            all_blocks_df.loc[curr_row[0], 'test_img'] = curr_target_img_dir
            
    # Save one sequence
    all_blocks_df['sub'] = 2*curr_sub
    all_blocks_df['TrialType'] = all_blocks_df['TrialType'].map(TrialType_dict)
    all_blocks_df['ifProbe'] = all_blocks_df['ifProbe'].map(ifProbe_dict)
    
    
    print(curr_prac_df[['TrialType', 'target_img_0']])
    write_json_file(all_blocks_df, curr_prac_df)
    
    
    # Save the mirror-image sequence (basically switch the images used in the 34 ms and 84 ms conditions [because these two were the ones showing strong effects during piloting])
    all_blocks_df_mirror = all_blocks_df.copy()
    all_blocks_df_mirror['FrameNum_old'] = all_blocks_df_mirror['FrameNum']
    FrameNum_dict = {2:5, 5:2,10:10}#switch the frame number
    all_blocks_df_mirror['FrameNum'] = all_blocks_df_mirror['FrameNum_old'].map(FrameNum_dict)
    all_blocks_df_mirror['sub'] = 2*curr_sub+1
    del all_blocks_df_mirror['FrameNum_old']
    write_json_file(all_blocks_df_mirror,curr_prac_df)






/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

   TrialType    target_img_0
17        NO    Filler/1.jpg
2        YES   Filler/14.jpg
15        NO   Filler/23.jpg
19        NO   Filler/36.jpg
7        YES   Filler/47.jpg
1        YES   Filler/56.jpg
9        YES   Filler/68.jpg
8        YES   Filler/80.jpg
18        NO   Filler/90.jpg
12        NO  Filler/100.jpg
11        NO  Filler/110.jpg
0        YES  Filler/121.jpg
3        YES  Filler/136.jpg
16        NO  Filler/146.jpg
14        NO  Filler/157.jpg
6        YES  Filler/171.jpg
4        YES  Filler/180.jpg
10        NO  Filler/190.jpg
5        YES  Filler/201.jpg
13        NO  Filler/210.jpg


/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

   TrialType    target_img_0
14        NO    Filler/1.jpg
7        YES   Filler/14.jpg
11        NO   Filler/23.jpg
16        NO   Filler/36.jpg
3        YES   Filler/47.jpg
6        YES   Filler/56.jpg
4        YES   Filler/68.jpg
5        YES   Filler/80.jpg
10        NO   Filler/90.jpg
2        YES  Filler/100.jpg
19        NO  Filler/109.jpg
15        NO  Filler/119.jpg
9        YES  Filler/136.jpg
13        NO  Filler/146.jpg
18        NO  Filler/157.jpg
8        YES  Filler/171.jpg
12        NO  Filler/180.jpg
1        YES  Filler/191.jpg
17        NO  Filler/201.jpg
0        YES  Filler/211.jpg


/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

...

   TrialType    target_img_0
0        YES    Filler/1.jpg
3        YES   Filler/12.jpg
1        YES   Filler/22.jpg
6        YES   Filler/30.jpg
17        NO   Filler/41.jpg
8        YES   Filler/54.jpg
13        NO   Filler/66.jpg
11        NO   Filler/79.jpg
15        NO   Filler/90.jpg
7        YES  Filler/100.jpg
19        NO  Filler/109.jpg
14        NO  Filler/119.jpg
12        NO  Filler/136.jpg
4        YES  Filler/148.jpg
5        YES  Filler/157.jpg
9        YES  Filler/166.jpg
18        NO  Filler/178.jpg
10        NO  Filler/190.jpg
16        NO  Filler/201.jpg
2        YES  Filler/211.jpg


/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

   TrialType    target_img_0
4        YES    Filler/1.jpg
15        NO   Filler/12.jpg
7        YES   Filler/23.jpg
5        YES   Filler/33.jpg
19        NO   Filler/43.jpg
2        YES   Filler/55.jpg
3        YES   Filler/67.jpg
12        NO   Filler/79.jpg
17        NO   Filler/90.jpg
13        NO  Filler/100.jpg
11        NO  Filler/110.jpg
9        YES  Filler/121.jpg
10        NO  Filler/136.jpg
14        NO  Filler/148.jpg
18        NO  Filler/159.jpg
6        YES  Filler/172.jpg
16        NO  Filler/181.jpg
0        YES  Filler/193.jpg
1        YES  Filler/203.jpg
8        YES  Filler/211.jpg


/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_blocks_df[curr_row[0], 'top'] = 0
/var/folders/f1/wkfpsl417m78r76gy_1p5hv80000gn/T/ipykernel_5086/4185557954.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

   TrialType    target_img_0
0        YES    Filler/1.jpg
14        NO   Filler/12.jpg
2        YES   Filler/23.jpg
9        YES   Filler/33.jpg
15        NO   Filler/43.jpg
16        NO   Filler/55.jpg
6        YES   Filler/68.jpg
19        NO   Filler/80.jpg
11        NO   Filler/91.jpg
18        NO  Filler/101.jpg
12        NO  Filler/111.jpg
1        YES  Filler/126.jpg
8        YES  Filler/138.jpg
10        NO  Filler/148.jpg
13        NO  Filler/159.jpg
7        YES  Filler/172.jpg
5        YES  Filler/181.jpg
4        YES  Filler/191.jpg
17        NO  Filler/201.jpg
3        YES  Filler/211.jpg


In [37]:
print(all_blocks_df.columns)


Index([   'TrialType',          'Pos',     'FrameNum',         'Dist',
                 'RE',     'blockNum',     'trialNum', 'target_img_0',
       'target_img_1', 'target_img_2',
       ...
         (375, 'top'),   (376, 'top'),   (377, 'top'),   (378, 'top'),
         (379, 'top'),   (380, 'top'),   (381, 'top'),   (382, 'top'),
         (383, 'top'),          'sub'],
      dtype='object', length=408)


In [38]:
curr_img_ind = 2
column_name = f'ProbeSize_{curr_img_ind}'  # Should be 'ProbeSize_2'
print(column_name)  # This should print 'ProbeSize_2'


ProbeSize_2
